In [7]:
import panel as pn
import pandas as pd
import numpy as np
import hvplot.pandas
import holoviews as hv
import param as pm 
import random
import math
import warnings
warnings.filterwarnings('ignore')
hv.extension('bokeh')


### Sigmoid
class AlgebraicSigmoid(pm.Parameterized):
    """
    A parameterized class to represent sigmoid curve.

    ...

    Attributes
    ----------
    l : number that adjusts the y-axis scale 
        default=20.8, bounds=(0, 100)
    s : number that adjusts the supply of tokens
        default=17, bounds=(1, 20)
    m : number that adjusts the slope
        default=21e6, bounds=(1, 21e6)
    k : number of tokens sold
        default=57300, bounds=(1, 1e5)
    steps : Integer 
        default=1000, bounds=(10, 10000)
    zoom : number that affects the scale of the presented view
        default=0.03, bounds=(0.01, 1)
    current_supply : number modeling the current tokens in circulation
        default=10000

    Methods
    -------
    f(x):
        Paramaterized Sigmoid Function.

    x():
        returns values for x axis scale based on m, zoom, steps.
   
    curve(x):
        Returns a dataframe containing the supply and price based on the X values
        supplied to f(x)

    collateral(x):
        Creates and returns the dataframe containing values that are less than the 
        current token supply

    view_curve():
        Return a holoviews line plot modeling the relationship between the supply
        and the price

    view_collateral():
        Returns a holoviews area plot that shows the relationship between the 
        supply and price

    view():
        Returns the two previous views over layed


    """

    l = pm.Number(20.8, bounds=(0, 100), precedence=-1)
    s = pm.Number(17, bounds=(1,20), precedence=-1)
    m = pm.Number(21e6, bounds=(1, 21e6), step=50000, precedence=-1)
    k = pm.Number(57300, bounds=(1, 1e5), step=100, precedence=-1)
    steps = pm.Integer(1000, bounds=(10,10000), step=10, precedence=-1)
    zoom = pm.Number(0.03, bounds=(0.01,1),step=0.01)
    current_supply = pm.Number(10000, step=1000)
    
    def __init__(self, **params):
        super(AlgebraicSigmoid, self).__init__(**params)
        self.param['current_supply'].bounds = (1,self.m*self.zoom)
    
    '''
    def f(self, x):
        """Parameterized Sigmoid Function"""
        self.param['current_supply'].bounds = (1,self.m*self.zoom)
        return self.k/(1+np.exp(-x*self.l/self.m+self.s))
    '''}
    
    def f(self, x_array: list = []) -> list:
        '''
        A function based on Molecule's medium article, their Algebraic
        Sigmoid Curve P(x)
        '''
        maximum_price: Union[int, float] =  50      # Letter 'a'
        supply_at_inflection: Union[int, float] =  12000000   # Letter 'b'
        slope_steepness: Union[int, float] = 120000000000     # Letter 'c', currently 120 billion


        a, b, c = maximum_price, supply_at_inflection, slope_steepness


        func = lambda x: a * ( (( x - b) / (sqrt(( x - b )**2 + c))) +1)

        #print(list(map(func, x_array)))
        return list(map(func, x_array))
    
    def x(self):
        x = np.linspace(0,self.m*self.zoom, self.steps)
        return x
    
    def curve(self, x):
        y = self.f(x)
        return pd.DataFrame(zip(x,y),columns=['supply','price'])
    
    def collateral(self, x):
        df = self.curve(x)
        return df[df['supply'] < self.current_supply]
    
    def view_curve(self):
        x = self.x()
        print(type(self.curve(x).hvplot.line(title='Bonding Curve', x='supply',y='price')))
        return self.curve(x).hvplot.line(title='Bonding Curve', x='supply',y='price')
    
    def view_collateral(self):
        x = self.x()
        print(type(self.collateral(x).hvplot.area(x='supply',y='price')))
        return self.collateral(x).hvplot.area(x='supply',y='price')
    
    def view(self):
        return self.view_curve()*self.view_collateral()



SyntaxError: unmatched '}' (1147375807.py, line 85)

In [8]:
sample = AlgebraicSigmoid()
pn.Column(sample,
          sample.view)

NameError: name 'AlgebraicSigmoid' is not defined

In [3]:
import numpy as np
from pprint import pprint
from math import sqrt
#from sympy import *

'''
a, b, c, x, p = symbols('a b c x p')

# Price function p(x) retruns the price for a single token at a specific supply 
algebraic_sigmoid_curve = Eq(
    a * ( ((x-b) / (sqrt((x-b)**2 + c))) +1),
    (p)
)
'''

# Collateral function C(x) returns the total capital needed to mint or burn a specified amount
# of tokens.
#collateral_function = Eq( a * (sqrt( ( b-x)**2 + c) + x ) - a * sqrt(b**2 + c ) , (c) )

def molecule_sigmoid(x_array: list = []) -> list:
    '''
    A function based on Molecule's medium article, their Algebraic
    Sigmoid Curve P(x)
    '''
    maximum_price: Union[int, float] =  50      # Letter 'a'
    supply_at_inflection: Union[int, float] = 250   # Letter 'b'
    slope_steepness: Union[int, float] = 5000     # Letter 'c'

    
    a, b, c = maximum_price, supply_at_inflection, slope_steepness
    
    
    func = lambda x: a * ( (( x - b) / (sqrt(( x - b )**2 + c))) +1)

    return list(map(func, x_array))

x = np.linspace(0, 500, 501).tolist()


In [4]:
x
y = molecule_sigmoid(x)
y

pd.DataFrame(zip(x,y), columns=['supply', 'price']).hvplot.line(title="Bonding Curve", x='supply', y='price')

NameError: name 'pd' is not defined